# SageMaker Pipelines workshop

This workshop is based on the [amazon sagemaker drift detection project](https://github.com/aws-samples/amazon-sagemaker-drift-detection) on github, available in the aws-samples repository. 

Amazon SageMaker Model Building Pipelines offers machine learning (ML) application developers and operations engineers the ability to orchestrate SageMaker jobs and author reproducible ML pipelines. It also enables them to deploy custom-build models for inference in real-time with low latency, run offline inferences with Batch Transform, and track lineage of artifacts. They can institute sound operational practices in deploying and monitoring production workflows, deploying model artifacts, and tracking artifact lineage through a simple interface, adhering to safety and best practice paradigms for ML application development.

The SageMaker Pipelines service supports a SageMaker Pipeline domain specific language (DSL), which is a declarative JSON specification. This DSL defines a directed acyclic graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that engineers and scientists are already familiar with.

## SageMaker Pipelines

SageMaker Pipelines supports the following activities, which are demonstrated in this notebook:

* Pipelines - A DAG of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional execution steps - A step that provides conditional execution of branches in a pipeline.
* Register model steps - A step that creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Create model steps - A step that creates a model for use in transform steps or later publication as an endpoint.
* Transform job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from a dataset, get inferences from large datasets, and run inference when a persistent endpoint is not needed.
* Parametrized Pipeline executions - Enables variation in pipeline executions according to specified parameters.

## Notebook Overview

This notebook shows how to:

* Define a set of Pipeline parameters that can be used to parametrize a SageMaker Pipeline.
* Define a Processing step that performs cleaning, feature engineering, and splitting the input data into train and test data sets.
* Define a Training step that trains a model on the preprocessed train data set.
* Define a Processing step that evaluates the trained model's performance on the test dataset.
* Define a Create Model step that creates a model from the model artifacts used in training.
* Define a Transform step that performs batch transformation based on the model that was created.
* Define a Register Model step that creates a model package from the estimator and model artifacts used to train the model.
* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes other steps.
* Define and create a Pipeline definition in a DAG, with the defined parameters and steps.
* Start a Pipeline execution and wait for execution to complete.
* Download the model evaluation report from the S3 bucket for examination.
* Start a second Pipeline execution.

We will consider 3 teams in this scenario:
* Engineering team - ML or data engineering
* Data science team - developing the models
* Dev ops team - automating and integrating the pipeline

## A SageMaker Pipeline

The pipeline that you create follows a typical machine learning (ML) application pattern of preprocessing, training, evaluation, model creation, batch transformation, and model registration:

![A typical ML Application pipeline](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-full.png)

### Dataset
we use the publicly available [New York City Taxi and Limousine Commission (TLC) Trip Record Data](https://registry.opendata.aws/nyc-tlc-trip-records-pds/). This TLC record data contains the yellow and green taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, tripc distances, itemized fares, rate types, payment types, and driver-reported passenger counts. The data used in the attached datasets were collected and provided to the NYC Taxi and Limousine Commission (TLC) by technology providers authorized under the Taxicab & Livery Passenger Enhancement Programs (TPEP/LPEP), please refer to [NYC documentation](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

### Importing the data and pre-processing
We import the data from the following S3 locations.
Please note that the data pre-processing script has been prepared and made availble in the pipeline folder- `preprocess.py`

In [2]:
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-tripfare'

# define the input data and input zone s3 location
input_source = f's3://{bucket}/{prefix}/input/'
input_data = input_source + 'data/green/'
input_zones = input_source + 'zones/'

# download the input data for the pipeline
!aws s3 cp --recursive 's3://nyc-tlc/trip data/' $input_data --exclude '*' --include 'green_tripdata_2018-1*'
!aws s3 cp 's3://nyc-tlc/misc/taxi_zones.zip' $input_zones

# confirm data is downloaded successfully
!aws s3 ls $input_data

%store input_data
%store input_zones

copy: s3://nyc-tlc/trip data/green_tripdata_2018-12.csv to s3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/data/green/green_tripdata_2018-12.csv
copy: s3://nyc-tlc/trip data/green_tripdata_2018-11.csv to s3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/data/green/green_tripdata_2018-11.csv
copy: s3://nyc-tlc/trip data/green_tripdata_2018-10.csv to s3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/data/green/green_tripdata_2018-10.csv
copy: s3://nyc-tlc/misc/taxi_zones.zip to s3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/zones/taxi_zones.zip
2021-11-11 03:56:47   63666398 green_tripdata_2018-10.csv
2021-11-11 03:56:47   58812608 green_tripdata_2018-11.csv
2021-11-11 03:56:47   61371115 green_tripdata_2018-12.csv
Stored 'input_data' (str)
Stored 'input_zones' (str)


## Preparation


Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [3]:
import json
import os

import boto3
import sagemaker
import sagemaker.session
from sagemaker import get_execution_role

from sagemaker.estimator import Estimator
from sagemaker.debugger import Rule, rule_configs
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.s3 import S3Uploader
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
    CacheConfig,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.utils import name_from_base


In [4]:
role = get_execution_role()

region = boto3.Session().region_name

sagemaker_session = sagemaker.Session()

bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-tripfare'
base_job_prefix = 'Demo-xgboost-tripfare'

### Define Parameters to Parametrize Pipeline Execution

Define Pipeline parameters that you can use to parametrize the pipeline. Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - represents a `str` Python type
* `ParameterInteger` - represents an `int` Python type
* `ParameterFloat` - represents a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes ( "PendingManualApproval" is the default).
* `input_data` - The S3 bucket URI location of the input trip data
* `input_zones` - The S3 bucket URI location of the input taxi zones data

![Define Parameters](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-1.png)

In [22]:
# checking if the input data and preprocessing script are already available, if not we need to define these variables
process_script = 'preprocess.py'
%store process_script
print(f'prcess script is available: {process_script}')
    
if 'input_data' not in locals() or 'input_zones' not in locals():
    input_data = f"s3://{bucket}/{prefix}/input/data/green/"
    input_zones = f"s3://{bucket}/{prefix}/input/zones/taxi_zones.zip"
    %store input_data
    %store input_zones
else:
    print(f'input data and input zones are available: \n {input_data} \n {input_zones}')

Stored 'process_script' (str)
prcess script is available: preprocess.py
input data and input zones are available: 
 s3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/data/green/ 
 s3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/zones/


In [23]:
# parameters for pipeline execution
     
input_data = ParameterString(
    name="InputDataUrl",
    default_value=input_data,
)
input_zones = ParameterString(
    name="InputZonesUrl",
    default_value=input_zones,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=2
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge"
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
model_output = ParameterString(
    name="ModelOutputUrl",
    default_value=f"s3://{bucket}/{prefix}/model",
)


In [24]:
# Create cache configuration (Unable to pass parameter for expire_after value)
cache_config = CacheConfig(enable_caching=True, expire_after="PT1H")

### Define a Processing Step for Feature Engineering

First, develop a preprocessing script that is specified in the Processing step.

In the SageMaker Processing lab, we have already created the `preprocess.py`, which contains the preprocessing script, and stored the path to the script. You can use the following cell to load the stored value.

The Processing step executes the script on the input data. The Training step uses the preprocessed training features and labels to train a model. The Evaluation step uses the trained model and preprocessed test features and labels to evaluate the model.

![Define a Processing Step for Feature Engineering](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-2.png)

Next, create an instance of an `SKLearnProcessor` processor and use that in our `ProcessingStep`.

You also specify the `framework_version` to use throughout this notebook.

Note the `processing_instance_type` and `processing_instance_count` parameters used by the processor instance.

In [25]:
# processing step for feature engineering
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name=f"{base_job_prefix}-sklearn-preprocess",
    sagemaker_session=sagemaker_session,
    role=role,
)

Finally, use the processor instance to construct a `ProcessingStep`, along with the input and output channels, and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` is the input data used in the step. This input data is used by the processor instance when it is run.

Also, note the `"train"`, `"validation"` and `"test"` named channels specified in the output configuration for the processing job. Step `Properties` can be used in subsequent steps and resolve to their runtime values at execution. Specifically, this usage is called out when you define the training step.

In [26]:
step_process = ProcessingStep(
    name="PreprocessData",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(
            source=input_data,
            destination="/opt/ml/processing/input/data",
            s3_data_distribution_type="ShardedByS3Key",
        ),
        ProcessingInput(
            source=input_zones,
            destination="/opt/ml/processing/input/zones",
            s3_data_distribution_type="FullyReplicated",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(
            output_name="validation", source="/opt/ml/processing/validation"
        ),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code=process_script,
# If you have created the Feature Group for this dataset in Lab 1, 
# you can use the feature group name here replace False to include the feature ingestion in the processing job
    job_arguments=['--fs_ing', "False",  
                   '--region', region,
                   '--bucket', bucket,
                  ],
    cache_config=cache_config,
)

### Define a Training Step to Train a Model

In this section, use Amazon SageMaker's [XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) to train on this dataset. Configure an Estimator for the XGBoost algorithm and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_output` so that it can be hosted later. 

The model path where the models from training will be saved is also specified.

Note the `training_instance_type` parameter may be used in multiple places in the pipeline. In this case, the `training_instance_type` is passed into the estimator.

![Define a Training Step to Train a Model](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-3.png)

In [27]:
# Define the XGBoost training report rules
# see: https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-training-xgboost-report.html
rules = [Rule.sagemaker(rule_configs.create_xgboost_report())]

# training step for generating model artifacts
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.2-2",
    py_version="py3",
    instance_type=training_instance_type,
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_output,
    base_job_name=f"{base_job_prefix}-train",
    sagemaker_session=sagemaker_session,
    role=role,
    disable_profiler=False,  # Profile processing job
    rules=rules,  # Report processing job
)

# Set some hyper parameters
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_train.set_hyperparameters(
    objective="reg:squarederror",
    num_round=100,
    early_stopping_rounds=10,
    max_depth=9,
    eta=0.2,
    gamma=4,
    min_child_weight=300,
    subsample=0.8,
)


Finally, use the estimator instance to construct a `TrainingStep` as well as the `properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that's executed when the pipeline invokes the pipeline execution. This is similar to an estimator's `fit` method in the Python SDK.

Pass in the `S3Uri` of the `"train"` output channel to the `TrainingStep`. Also, use the other `"validation"` output channel for model evaluation in the pipeline. The `properties` attribute of a Pipeline step matches the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [28]:
step_train = TrainingStep(
    name="TrainModel",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
    cache_config=cache_config,

)

### Define a Model Evaluation Step to Evaluate the Trained Model

First, develop an evaluation script that is specified in a Processing step that performs the model evaluation.

After pipeline execution, you can examine the resulting `evaluation.json` for analysis.

The evaluation script uses `xgboost` to do the following:

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including mae, mse, rmse and r2 metrics.
* Save the evaluation report to the evaluation directory.

![Define a Model Evaluation Step to Evaluate the Trained Model](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-4.png)

In [29]:
%%writefile evaluation.py
"""Evaluation script for measuring mean squared error."""
import json
import logging
import pathlib
import pickle
import tarfile

import numpy as np
import pandas as pd
import xgboost
import glob

from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

if __name__ == "__main__":
    logger.debug("Starting evaluation.")
    
    import os
    for file in os.listdir("/opt/ml/processing/model"):
        logger.info(file)
        
    model_path = "/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    
    for file in os.listdir("/opt/ml/processing/model"):
        logger.info(file)
        
    logger.debug("Loading xgboost model.")
    model = pickle.load(open("xgboost-model", "rb"))

    logger.debug("Reading test data.")
    test_path = "/opt/ml/processing/test/"
    all_files = glob.glob(test_path + "/*.csv")
    tmp = []
    for filename in all_files:
        df_tmp = pd.read_csv(filename, index_col=None, header=None)
        tmp.append(df_tmp)

    df = pd.concat(tmp, axis=0, ignore_index=True)

    logger.debug("Reading test data.")
    y_test = df.iloc[:, 0].values
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = xgboost.DMatrix(df.values)

    logger.info("Performing predictions against test data.")
    predictions = model.predict(X_test)

    # See the regression metrics
    # see: https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html
    logger.debug("Calculating metrics.")
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = sqrt(mse)
    r2 = r2_score(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mae": {
                "value": mae,
                "standard_deviation": std,
            },
            "mse": {
                "value": mse,
                "standard_deviation": std,
            },
            "rmse": {
                "value": rmse,
                "standard_deviation": std,
            },
            "r2": {
                "value": r2,
                "standard_deviation": std,
            },
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    logger.info("Writing out evaluation report with rmse: %f", rmse)
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting evaluation.py


Next, create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [30]:
# processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name=f"{base_job_prefix}/script-eval",
    sagemaker_session=sagemaker_session,
    role=role,
)


Use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Specifically, the `S3ModelArtifacts` from the `step_train` `properties` and the `S3Uri` of the `"test"` output channel of the `step_process` `properties` are passed into the inputs. The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [31]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)
step_eval = ProcessingStep(
    name="EvaluateModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation", source="/opt/ml/processing/evaluation"
        ),
    ],
    code="evaluation.py",
    property_files=[evaluation_report],
    cache_config=cache_config,

)

### Define a Create Model Step to Create a Model

In order to perform batch transformation using the example model, create a SageMaker model. 

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

![Define a Create Model Step and Batch Transform to Process Data in Batch at Scale](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-5.png)

In [32]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.model import Model


model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.xlarge",
)
step_create_model = CreateModelStep(
    name="TripFareCreateModel",
    model=model,
    inputs=inputs,
)

### Define a Transform Step to Perform Batch Transformation

Now that a model instance is defined, create a `Transformer` instance with the appropriate model type, compute instance type, and desired output S3 URI.

Specifically, pass in the `ModelName` from the `CreateModelStep`, `step_create_model` properties. The `CreateModelStep` `properties` attribute matches the object model of the [DescribeModel](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeModel.html) response object.

In [33]:
from sagemaker.transformer import Transformer

batch_output = f"s3://{bucket}/{prefix}/batchtransform"
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=batch_output,
    accept="text/csv",
    assemble_with="Line"
)

Pass in the transformer instance and the `TransformInput` with the `batch_data` pipeline parameter defined earlier. More details of the `TransformInput` can be found [here](https://github.com/aws/sagemaker-python-sdk/blob/2594ffb3eaefaf55936b71ea0c38442135223602/src/sagemaker/inputs.py#L140). We use `input_filter` to filter out the first column, which is the label column, in the test data. We also set the batch job to join the input source with the batch job output. Please check the document about [associating inference with input record](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) for more details

In [34]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="TripFareTransform",
    transformer=transformer,
    inputs=TransformInput(
        data=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
        content_type="text/csv", 
        split_type="Line",
        input_filter="$[1:]",
        join_source="Input",
        ),
    cache_config=cache_config,
)


### Define a Register Model Step to Create a Model Package

Use the estimator instance specified in the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a model package. A model package is a reusable model artifacts abstraction that packages all ingredients required for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. A model package group can be created for a specific ML business problem, and new versions of the model packages can be added to it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker pipeline so that model package versions can be added to the group for every SageMaker Pipeline run.

The construction of `RegisterModel` is similar to an estimator instance's `register` method in the Python SDK.

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Note that the specific model package group name provided in this notebook can be used in the model registry and CI/CD work with SageMaker Projects.

In [35]:
# register model step that will be conditionally executed
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
                "S3Uri"
            ]
        ),
        content_type="application/json",
    )
)
model_package_group_name = "TripFareDemoModelPackage"
step_register = RegisterModel(
    name="RegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.t2.large", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

### Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry

In this step, the model is registered only if the accuracy of the model, as determined by the evaluation step `step_eval`, exceeded a specified value. A `ConditionStep` enables pipelines to support conditional execution in the pipeline DAG based on the conditions of the step properties. 

In the following section, you:

* Define a `ConditionLessThanOrEqualTo` on the accuracy value found in the output of the evaluation step, `step_eval`.
* Use the condition in the list of conditions in a `ConditionStep`.
* Pass the `CreateModelStep` and `TransformStep` steps, and the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`, which are only executed, if the condition evaluates to `True`.

![Define a Condition Step to Check Accuracy and Conditionally Execute Steps](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-6.png)

In [36]:
# condition step for evaluating model quality and branching execution
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.rmse.value",
    ),
    right=8.0,
)
step_cond = ConditionStep(
    name="CheckEvaluation",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[],
)

### Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All of the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the _data dependency_ DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

![Define a Pipeline of Parameters, Steps, and Conditions](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-7.png)

In [37]:
# pipeline instance
pipeline = Pipeline(
    name="test-Demo-pipeline-train",
    parameters=[
        input_data,
        input_zones,
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        model_approval_status,
        model_output,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
    sagemaker_session=sagemaker_session,
)

#### (Optional) Examining the pipeline definition

The JSON of the pipeline definition can be examined to confirm the pipeline is well-defined and the parameters and step properties resolve correctly.

In [38]:
import json


definition = json.loads(pipeline.definition())
definition

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'InputDataUrl',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/data/green/'},
  {'Name': 'InputZonesUrl',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/input/zones/'},
  {'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 2},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'ModelOutputUrl',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-southeast-2-571660658801/sagemaker/DEMO-xgboost-tripfare/model'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'G

### Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The role passed in will be used by the Pipeline service to create all the jobs defined in the steps.

In [39]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:pipeline/test-demo-pipeline-train',
 'ResponseMetadata': {'RequestId': 'a2534a0b-fb1a-4b55-b05e-8b7af4526465',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a2534a0b-fb1a-4b55-b05e-8b7af4526465',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Thu, 11 Nov 2021 03:58:54 GMT'},
  'RetryAttempts': 0}}

### Pipeline Operations: Examining and Waiting for Pipeline Execution

Describe the pipeline execution.

In [40]:
execution = pipeline.start()

In [41]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:pipeline/test-demo-pipeline-train',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:pipeline/test-demo-pipeline-train/execution/0yaufvm95qt5',
 'PipelineExecutionDisplayName': 'execution-1636603150434',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 11, 11, 3, 59, 10, 370000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 11, 11, 3, 59, 10, 370000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:user-profile/d-havwgmpljtgq/machinelearning',
  'UserProfileName': 'machinelearning',
  'DomainId': 'd-havwgmpljtgq'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:user-profile/d-havwgmpljtgq/machinelearning',
  'UserProfileName': 'machinelearning',
  'DomainId': 'd-havwgmpljtgq'},
 'ResponseMetadata': {'RequestId': '0364dcaf-c79a-4bf4-aef6-f133801fd3e6',
  'HTTPStatus

In [42]:
# Wait for the execution to complete.
execution.wait()

### Examining the Evalution

Examine the resulting model evaluation after the pipeline completes. Download the resulting `evaluation.json` file from S3 and print the report.

In [43]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
pprint(json.loads(evaluation_json))

{'regression_metrics': {'mae': {'standard_deviation': 7.540204273464937,
                                'value': 4.384029849601721},
                        'mse': {'standard_deviation': 7.540204273464937,
                                'value': 56.8608452528415},
                        'r2': {'standard_deviation': 7.540204273464937,
                               'value': 0.6260861618077611},
                        'rmse': {'standard_deviation': 7.540204273464937,
                                 'value': 7.540613055504274}}}


### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [44]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

{'StepName': 'PreprocessData', 'StartTime': datetime.datetime(2021, 11, 11, 3, 59, 11, 251000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 11, 11, 4, 3, 46, 872000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:processing-job/pipelines-0yaufvm95qt5-preprocessdata-mlngdgssc1'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...11-03-58-54-436/input/code/preprocess.py,Input,DataSet,ContributedTo,artifact
1,s3://...maker/DEMO-xgboost-tripfare/input/zones/,Input,DataSet,ContributedTo,artifact
2,s3://.../DEMO-xgboost-tripfare/input/data/green/,Input,DataSet,ContributedTo,artifact
3,78335...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
4,s3://...oces-2021-11-11-03-58-51-338/output/test,Output,DataSet,Produced,artifact
5,s3://...021-11-11-03-58-51-338/output/validation,Output,DataSet,Produced,artifact
6,s3://...ces-2021-11-11-03-58-51-338/output/train,Output,DataSet,Produced,artifact


{'StepName': 'TrainModel', 'StartTime': datetime.datetime(2021, 11, 11, 4, 3, 47, 988000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 11, 11, 4, 9, 17, 116000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:training-job/pipelines-0yaufvm95qt5-trainmodel-icxy63zhif'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...021-11-11-03-58-51-338/output/validation,Input,DataSet,ContributedTo,artifact
1,s3://...ces-2021-11-11-03-58-51-338/output/train,Input,DataSet,ContributedTo,artifact
2,78335...-2.amazonaws.com/sagemaker-xgboost:1.2-2,Input,Image,ContributedTo,artifact
3,s3://...rainModel-icxY63zHiF/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'EvaluateModel', 'StartTime': datetime.datetime(2021, 11, 11, 4, 9, 18, 121000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 11, 11, 4, 13, 26, 692000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:processing-job/pipelines-0yaufvm95qt5-evaluatemodel-0xs7f5j6ut'}}}


...

,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...rainModel-icxY63zHiF/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,78335...-2.amazonaws.com/sagemaker-xgboost:1.2-2,Input,Image,ContributedTo,artifact
2,tripfaredemomodelpackage-1-PendingManualApprov...,Input,Approval,ContributedTo,action
3,TripFareDemoModelPackage-1636604010-aws-model-...,Output,ModelGroup,AssociatedWith,context


{'StepName': 'TripFareCreateModel', 'StartTime': datetime.datetime(2021, 11, 11, 4, 13, 29, 410000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 11, 11, 4, 13, 30, 883000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:model/pipelines-0yaufvm95qt5-tripfarecreatemodel-9b5pehstuj'}}}


None

{'StepName': 'TripFareTransform', 'StartTime': datetime.datetime(2021, 11, 11, 4, 13, 31, 465000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 11, 11, 4, 18, 0, 795000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:571660658801:transform-job/pipelines-0yaufvm95qt5-tripfaretransform-micnfbzl0u'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...rainModel-icxY63zHiF/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,78335...-2.amazonaws.com/sagemaker-xgboost:1.2-2,Input,Image,ContributedTo,artifact
2,s3://...oces-2021-11-11-03-58-51-338/output/test,Input,DataSet,ContributedTo,artifact
3,s3://...ker/DEMO-xgboost-tripfare/batchtransform,Output,DataSet,Produced,artifact


### Parametrized Executions

You can run additional executions of the pipeline and specify different pipeline parameters. The parameters argument is a dictionary containing parameter names, and where the values are used to override the defaults values.

Based on the performance of the model, you might want to kick off another pipeline execution on a compute-optimized instance type and set the model approval status to "Approved" automatically. This means that the model package version generated by the `RegisterModel` step is automatically ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [175]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [176]:
execution.wait()

In [177]:
execution.list_steps()

[{'StepName': 'TripFareTransform',
  'StartTime': datetime.datetime(2021, 11, 11, 3, 28, 24, 231000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 11, 11, 3, 33, 15, 746000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:790767156351:transform-job/pipelines-rt6j1qe4jiil-tripfaretransform-g9tz6qd42t'}}},
 {'StepName': 'TripFareCreateModel',
  'StartTime': datetime.datetime(2021, 11, 11, 3, 28, 17, 206000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 11, 11, 3, 28, 18, 419000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:790767156351:model/pipelines-rt6j1qe4jiil-tripfarecreatemodel-km1s12yjow'}}},
 {'StepName': 'RegisterModel',
  'StartTime': datetime.datetime(2021, 11, 11, 3, 28, 17, 206000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 11, 11, 3, 28, 21, 72000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Met